In [1]:
# Model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
import torch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
# Vector stores
import fitz  # PyMuPDF
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter, MarkdownHeaderTextSplitter
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader ,UnstructuredPDFLoader
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_teddynote.retrievers import KiwiBM25Retriever, OktBM25Retriever
from langchain.docstore.document import Document
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import unicodedata
import pymupdf4llm
import time
import re
from konlpy.tag import Okt
from pdf2image import convert_from_path
import pytesseract
from konlpy.tag import Kkma
# etc
import os
import pandas as pd
from tqdm import tqdm
import unicodedata
import logging
from PyPDF2 import PdfReader
import json
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
print(device)

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
from langchain_unstructured import UnstructuredLoader
# intfloat/multilingual-e5-small
# jhgan/ko-sroberta-multitask
def get_embedding():
    embeddings = HuggingFaceEmbeddings(
        model_name='jhgan/ko-sroberta-multitask',
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True})
    return embeddings

def normalize_string(s):
    try:
        normalized = unicodedata.normalize('NFC', s)
        return normalized.encode('utf-8', errors='replace').decode('utf-8')
    except Exception:
        return s
def clean_text(text):
    text = text.replace("�", " ").replace("", " ") # 잘못된 인코딩 문자 제거
    text = ' '.join(text.split())  # 여러 공백을 하나로 줄임
    return text


In [3]:
def get_docs(pdf_path):
    documents = []
    failed_pages = []  # 실패한 페이지를 추적하기 위한 리스트

    try:
        # 페이지 단위로 pymupdf4llm 사용
        md_read = to_markdown(pdf_path, page_chunks=True)
        total_pages = len(md_read)

        for page_data in md_read:
            try:
                page_number = page_data.get('metadata', {}).get('page', None)
                if page_number is None:
                    raise ValueError("Page number missing in metadata")

                text = clean_text(normalize_string(page_data.get('text', '')))
                if not text:
                    raise ValueError("Empty text")  # 텍스트가 비어 있으면 예외 발생

                # 헤더 정보를 메타데이터에 포함
                headers = {
                    "Header 1": page_data.get('metadata', {}).get('header1', ''),
                    "Header 2": page_data.get('metadata', {}).get('header2', ''),
                    "Header 3": page_data.get('metadata', {}).get('header3', '')
                }

                metadata = {
                    "file_path": pdf_path,
                    "page_number": page_number,
                    "total_pages": total_pages,
                    "tables": page_data.get('tables', []),
                    **headers  # 헤더 정보를 메타데이터에 포함
                }

                document = Document(page_content=text, metadata=metadata)
                documents.append(document)

            except Exception as e:
                logging.warning(f"Failed to process page {page_number} with pymupdf4llm: {e}")
                failed_pages.append(page_number)

    except Exception as e:
        logging.error(f"Failed to read PDF with pymupdf4llm: {e}")
        # pymupdf4llm 전체 실패 시 모든 페이지를 PyMuPDF로 처리하도록 설정
        failed_pages = list(range(1, len(fitz.open(pdf_path)) + 1))

    # 실패한 페이지에 대해 PyMuPDF로 재처리
    if failed_pages:
        try:
            doc = fitz.open(pdf_path)
            for page_num in failed_pages:
                page = doc.load_page(page_num - 1)  # 페이지 번호는 0부터 시작하기 때문에 -1
                text = page.get_text("text")
                if text:
                    text = clean_text(normalize_string(text))
                    metadata = {
                        "file_path": pdf_path,
                        "page_number": page_num,
                        "total_pages": doc.page_count
                    }
                    document = Document(page_content=text, metadata=metadata)
                    documents.append(document)

        except Exception as e:
            logging.error(f"Failed to read PDF with PyMuPDF: {e}")

    return documents

def is_markdown(text):
    # 간단히 마크다운 패턴을 확인하는 함수
    return any(header in text for header in ['# ', '## ', '### ', '#### ', '- ', '* '])

def chunk_documents(docs):
    markdown_headers = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
    
    # 마크다운 헤더 스플리터 설정
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=markdown_headers,
        strip_headers=False  # 헤더를 유지
    )
    
    # 기본 텍스트 스플리터 설정
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=600,
        chunk_overlap=60,
        separators=["\n\n", ".", "?", "!", "\n", "\t"]
    )
    
    chunks = []

    for doc in docs:
        text = doc.page_content
        metadata = doc.metadata

        # 문서 내에 헤더 정보가 있을 때
        if any(metadata.get(header) for header in ["Header 1", "Header 2", "Header 3"]):
            # 마크다운 헤더 기준으로 청크 분할
            md_header_splits = markdown_splitter.split_text(text)
            for md_split in md_header_splits:
                # 마크다운 스플리터로 생성된 청크에 대해 추가 분할 적용
                smaller_chunks = text_splitter.split_documents([Document(page_content=md_split.page_content, metadata={**metadata, **md_split.metadata})])
                chunks.extend(smaller_chunks)
        else:
            # 마크다운이 아닌 경우 일반적인 방식으로 분할
            text_chunks = text_splitter.split_text(text)
            chunks.extend([Document(page_content=chunk, metadata=metadata) for chunk in text_chunks])

    return chunks

def make_db(df):
    documents = []
    pdf_files = df['Source_path'].unique()

    # tqdm으로 파일 처리 진행 상황 표시
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(get_docs, pdf_files), total=len(pdf_files), desc="Processing PDFs"))

    for result in results:
        documents.extend(result)

    # 정규화
    for doc in documents:
        doc.page_content = normalize_string(doc.page_content)

    # 표가 망가지지 않도록 청크 분할 처리
    chunks = chunk_documents(documents)
    print(f"Total number of chunks: {len(chunks)}")

    faiss = FAISS.from_documents(chunks, embedding=get_embedding())
    bm = OktBM25Retriever.from_documents(chunks)

    return faiss, bm

In [4]:
def fewshot_db(df):
    df = df.drop('SAMPLE_ID', axis=1)
    df = df.drop('Source_path', axis=1)
    df = df.to_dict(orient='records')
    print("Loaded Fewshot Set:", len(df))
    to_vectorize = ["\n\n".join(normalize_string(value) for value in example.values()) for example in df]
    faiss = FAISS.from_texts(to_vectorize, embedding=get_embedding())
    # bm = BM25Retriever.from_texts(to_vectorize)
    # knn = KNNRetriever.from_texts(to_vectorize, embeddings=get_embedding())
    return faiss

In [5]:
train_df = pd.read_csv('train.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8')

In [6]:
def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for i, doc in enumerate(docs):
        #context += f"Document {i+1}\n"
        doc.page_content = doc.page_content.replace("{", "(")
        doc.page_content = doc.page_content.replace("}", ")")
        
        context += doc.page_content
        context += '\n\n'
    return context.strip()

In [7]:
def setup_llm_pipeline(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"):
    # 토크나이저 로드 및 설정
        # 양자화 설정 적용
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,low_cpu_mem_usage=True)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=1024,
        eos_token_id = terminators,
        pad_token_id = tokenizer.eos_token_id
    )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm
# ghost-x/ghost-8b-beta-1608
# OpenBuddy/openbuddy-llama3.1-8b-v22.3-131k
llm = setup_llm_pipeline(model_id="OpenBuddy/openbuddy-llama3.1-8b-v22.3-131k")

In [ ]:
from collections import Counter
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())
    
    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):
    
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    
    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
    
    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)
    
    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }

In [ ]:
def extract_answer(response):
    # AI: 로 시작하는 줄을 찾아 그 이후의 텍스트만 추출
    lines = response.split('\n')
    for line in lines:
        line = line.replace('**', '')
        if line.startswith('Answer:'):
            return line.replace('Answer:', '').strip()
        if line.startswith('assistant:'):
            return line.replace('assistant:', '').strip()
    return response.strip()  # AI: 를 찾지 못한 경우 전체 응답을 정리해서 반환

def rerun(question,context,answer,llm,num_repeat):
    full_template = ""
    full_template += """<|role|>system<|says|>당신은 답변을 검증하는 챗봇입니다. 질문과 문맥, 이전 답변을 참고해서 지시사항을 따르세요.<|end|>"""
    full_template += f"""<|role|>user<|says|>Question: {question} \n\nContexts: {context} \n\nPrevious Answer: {answer} \n\n"""
    full_template += """{input}<|end|>"""
    full_template += """<|role|>assistant<|says|>"""
    
    prompt = PromptTemplate(template=full_template)
    chain = (
    {
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
    )
    return chain.invoke("핵심 단어를 살려서 한 문장으로 요약하세요. 만약 한 문장이라면 그대로 출력하세요.")
    

def run (test,dataset,llm,verbose=False):
    results = []
    for i, row in (dataset.iterrows()):
        full_template = ""
        full_template += """<|role|>system<|says|>
당신(assistant)은 유용한 금융 정보 QnA 챗봇입니다.
질문에 모두 답해야 하며, 답변은 반드시 문맥 정보를 활용해야합니다. 
단위는 문맥 정보에 같이 포함되어있습니다.
객관적이고 공식적인 문체를 사용하세요.
어떤 서론이나 배경 설명 없이 바로 핵심 내용을 전달해주세요. 
한 문장으로 작성하세요.
<|eot_id|>
"""
        question = row['Question']          
        # full_template += """ """
        contexts = test.invoke(normalize_string(question))
        contexts = format_docs(contexts)
        full_template += """<|role|>user<|says|>Question: {input}\n\n"""
        full_template += f"""Contexts: {contexts}<|end|>"""
        full_template += """<|role|>assistant<|says>"""
        
        prompt = PromptTemplate(template=full_template, input_variables=["input"])
        qa_chain = (
        {
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )

        answer = qa_chain.invoke(input=question)
        answer = extract_answer(answer)
        lines = answer.split('\n')
        if  len(lines) > 1:
            previous = answer
            try:
                before = calculate_f1_score(row['Answer'],answer)[2]
            except:
                before = None
            answer = rerun(question=question,
                           context=contexts,
                           answer=answer,
                           llm=llm,
                           num_repeat=1)
        answer = extract_answer(answer)
        results.append({
            "Question": question,
            "Answer": answer,
            "Source": row['Source']
        })
        if verbose:
            print(f"{i}/{len(dataset)}")
            print("Question: ", question, end=" | ")
            print("Context Number |",len(contexts))
            try:
                print(calculate_f1_score(row['Answer'],answer)[2],end=" | ")
            except:
                pass
            print("Answer: ", results[-1]['Answer'])
            try:
                print("Before: ",before," | ",previous)  
                
                previous = None
                before = None
            except:
                pass
            
            try:
                print("REAL Answer: ",row['Answer'])
            except:
                pass
            
            print()

    return results

In [ ]:
from sklearn.model_selection import KFold
import copy

weight = [0.5,0.5]
train_faiss_db, train_bm_retrievier = make_db(train_df) 

train_k = 3
train_bm_retrievier.k = train_k
# knn_retriever.k = train_k
train_faiss_retriever = train_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':train_k} )
train_ensemble_retriever = EnsembleRetriever(
    retrievers=[train_bm_retrievier, train_faiss_retriever], weights=weight
)


fewshot_k = 3

k_folds = 4
fold_results = []
kf = KFold(n_splits=k_folds, shuffle=True, random_state=52)
for fold, (train_index, val_index) in enumerate(kf.split(train_df)):
    fold_result = []
    train_set = train_df.iloc[train_index]
    val_set = train_df.iloc[val_index]

    pred = run(train_ensemble_retriever, val_set, llm, verbose=True)
    result = pd.DataFrame()
    result['pred'] = [result['Answer'] for result in pred]
    val_set.index = range(len(val_set))
    result['gt'] = val_set['Answer']
        
    result = calculate_average_f1_score(result['gt'], result['pred'])
    print(result)
    fold_results.append(result)
    break

ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
Processing PDFs:   0%|          | 0/16 [00:00<?, ?it/s]ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4

Total number of chunks: 2044


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\llama\modeling_llama.py:660: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


1/124
Question:  2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요? | Context Number | 1689
0.07909604519774012 | Answer:  2024년 중앙정부 예산은 일반회계와 특별회계로 구성되며, 특별회계는 21개로 다양한 정책 분야를 지원합니다. 각 특별회계의 예산 배분 비중을 분석하여 정책 우선순위를 파악하고 예산 분배 현황을 살펴보겠습니다. 국토는 1조 9천억원으로, 국가 인프라 구축 및 교통 시스템 확충에 집중적으로 투입됩니다. 환경은 1조 6천억원으로, 지속가능한 도시 개발, 에너지 효율성 증대, 환경 보호 등에 중점을 두고 예산을 배분합니다. 산업은 1조 4천억원으로, 첨단산업 육성, 기업 지원, 혁신 기술 개발에 투자하며, 2024년 예산에서 산업 분야는 1조 4천억원으로 확보됩니다. 사회복지 분야는 1조 1천억원으로, 국민의 기본적인 사회보장 및 복지 증진을 위해 예산이 집중됩니다. 국방은 8조 6천억원으로, 국가 안보 강화, 군사력 현대화, 국방력 확보를 위한 투자를 주도합니다. 교육은 7조 8천억원으로, 교육 인프라 확충, 교사 양성, 교육 수준 향상 등 교육 분야 발전을 위한 예산이 확보됩니다. 농림어업은 5조 3천억원으로, 농업 지원, 어업 발전, 농촌 지역 발전을 위한 투자로 구성됩니다. 외교통상은 5조 1천억원으로, 국제 협력 강화, 외교 정책 수행, 경제협력 확대에 기여합니다. 보건복지은 4조 5천억원으로, 국민의 건강 증진, 의료 서비스 확충, 보건 정책 추진 등에 중점을 두고 예산을 배분합니다. 문화체육관광은 3조 5천억원으로, 문화 예술 발전, 스포츠 육성, 관광 산업 활성화에 투자하며, 2024년 예산에서 문화 분야는 3조 5천억원으로 확보됩니다. 법제도는 2조 8천억원으로, 법률 제정 및 개정, 법률 시행 지원, 법제도 개혁 등에 예산을 집중적으로 지원합니다. 행정안전은 2조 7천억원으로, 행정 시스템 개선, 공공 서비스 질 향상, 지역 발전 지원 등에 투자하며, 2024년 

In [ ]:
# from save_module import save


# weight = [0.5,0.5]
# test_faiss_db, test_bm_retrievier = make_db(test_df)

# test_k = 3
# test_bm_retrievier.k = test_k
# #test_knn_retriever.k = test_k
# test_faiss_retriever = test_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':test_k} )
# test_ensemble_retriever = EnsembleRetriever(
#     retrievers=[test_bm_retrievier, test_faiss_retriever], weights=weight
# )


# results = run(test_ensemble_retriever, test_df, llm, verbose=True)
# save(results)

ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
ERROR:root:Failed to read PDF with pymupdf4llm: name 'to_markdown' is not defined
Processing PDFs: 100%|██████████| 9/9 [00:00<00:00, 59.17it/s]


Total number of chunks: 197


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\llama\modeling_llama.py:660: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


0/98
Question:  2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요? | Context Number | 2455
Answer:  2022년 혁신창업사업화자금(융자)의 예산은 2,007,800만원입니다.

1/98
Question:  중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요? | Context Number | 3302
Answer:  중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의 창업을 활성화하고 고용창출을 도모하며, 중소기업이 보유한 우수 기술의 개발과 기술기반 중소기업을 육성하기 위함입니다.

2/98
Question:  중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요? | Context Number | 2895
Answer:  중소벤처기업부의 혁신창업사업화자금(융자) 사업의 법적 근거는 중소기업진흥에 관한 법률 제66조, 제66조, 제67조, 제74조입니다.

3/98
Question:  2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요? | Context Number | 2128
Answer:  2010년에 신규 지원된 혁신창업사업화자금은 재창업자금(실패 경영인에 대한 재기 지원)입니다.

4/98
Question:  혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요? | Context Number | 2061
Answer:  2020년에 신규 지원된 혁신창업사업화자금은 2,300,000 만원입니다.

5/98
Question:  재창업자금이 재도약지원자금으로 이관된 연도는 언제인가요? | Context Number | 2292
Answer:  2023

6/98
Question:  창업기반지원과 신청 대상이 중복인 자금이 어떤 것이며, 이 자금이 폐지된 연도는 언제인가요? | Context Number | 3627
Answer:  창업기반지원과 신청 대상이 중복인 자금은 일자리창출촉진자금입니다.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


8/98
Question:  혁신창업사업화자금(융자) 사업 집행절차는 어떻게 되나요? | Context Number | 3021
Answer:  

혁신창업사업화자금(융자)의 집행절차는 사업신청, 사업신청서심사, 사업승인, 융자대출, 융자대출 후 관리의 5단계로 이루어집니다.
Before:  None  |  혁신창업사업화자금(융자)의 집행절차는 다음과 같습니다.

1. 사업신청: 혁신창업사업화자금(융자) 지원을 희망하는 중소기업은 중소벤처기업부 중소기업정책실에 사업신청서를 제출해야 합니다.
2. 사업신청서심사: 중소벤처기업부 중소기업정책실은 신청서를 심사하여 사업신청서를 접수한 후 1주일 이내에 심사 결과를 통보합니다.
3. 사업승인: 사업신청서가 심사 결과로 승인된 경우, 중소벤처기업부 중소기업정책실은 사업자에게 사업승인통보서를 발급하고, 사업자금의 융자신청서를 제출받아 융자대출을 승인합니다.
4. 융자대출: 중소벤처기업부 중소기업정책실은 융자신청서를 심사하여 융자대출을 승인한 후, 융자금을 지급합니다.
5. 융자대출 후 관리: 융자대출 후 중소벤처기업부 중소기업정책실은 융자대출 후 관리를 위한 사업자금의 사용 및 관리를 감독합니다.

9/98
Question:  부모급여 지원 사업의 목적은 무엇인가요? | Context Number | 2412
Answer:  부모급여 지원 사업의 목적은 출산 및 양육으로 손실되는 소득을 보전하고, 주 양육자의 직접 돌봄이 중요한 아동 발달의 특성에 따라 영아기 돌봄을 두텁게 지원하기 위해 부모급여 지급입니다.
Before:  None  |  None

10/98
Question:  부모급여(영아수당)의 2024년 확정된 예산은 몇백만원인가요? | Context Number | 4275
Answer:  2024년 부모급여(영아수당)의 확정된 예산은 2,888,694백만원입니다.
Before:  None  |  None

11/98
Question:  부모급여 지원 사업은 어떤 법령상 근거를 갖고 추진되고 있나요? | Con